# Imports and Installs

In [0]:
# install latest version of polyglot language detection library for stretch
!pip install -U git+https://github.com/aboSamoor/polyglot.git@master

In [0]:
import pandas as pd
import numpy as np 
import pandas_profiling 
from sklearn import preprocessing # for category encoder
from polyglot.detect import Detector # for language detection stretch
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
# much more efficient for larger files like Nearest Neighbors which the model
import joblib

# EDA & PROFILE REPORT

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/aguilargallardo/DS-Unit-2-Applied-Modeling/master/data/SpotifyFeatures.csv')

In [0]:
# pandas_profiling.ProfileReport(df)

# Data Engineering Language Detection (Stretch, not working yet)

In [0]:
# # test of polyglot detector

# arabic_text = u"""
# أفاد مصدر امني في قيادة عمليات صلاح الدين في العراق بأن " القوات الامنية تتوقف لليوم
# الثالث على التوالي عن التقدم الى داخل مدينة تكريت بسبب
# انتشار قناصي التنظيم الذي يطلق على نفسه اسم "الدولة الاسلامية" والعبوات الناسفة
# والمنازل المفخخة والانتحاريين، فضلا عن ان القوات الامنية تنتظر وصول تعزيزات اضافية ".
# """

# detector = Detector(arabic_text)
# print(detector.language)

In [0]:
"""
   TODO: figure out UTF-8 ERROR or just do try catch exception and return 
   unknown or something
   below code does not work
"""

# df.track_name = df.track_name.astype(str)
# df['poly_obj'] = df.track_name.apply(lambda x: Detector(x, quiet=True))
# df['Track-lang'] = df['poly_obj'].apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
# df['Track-LangConfidence'] = df['poly_obj'].apply( lambda x: x.language.confidence)


# DATA ENGINEERING CATEGORY ENCODING OF OBJECT FEATURES
note: might need to create another column of encoded languages
resources below: https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/


In [0]:
df = df.dropna()

In [0]:
df.key.value_counts()

In [0]:
time_sig_encoding = { '0/4' : 0, '1/4' : 1, 
                     '3/4' : 3, '4/4' : 4,
                     '5/4' : 5}

In [0]:
key_encoding = { 'A' : 0, 'A#' : 1, 'B' : 2,
                'C' : 3,  'C#' : 4,  'D' : 5,
                'D#' : 6, 'E' : 7, 'F' : 8,
                'F#' : 9, 'G' : 10, ' G#' : 11 }  

In [0]:
mode_encoding = { 'Major':0, 'Minor':1}

In [0]:
df['mode'].value_counts()

In [0]:
df['key'] = df['key'].map(key_encoding)

In [0]:
df['time_signature'] = df['time_signature'].map(time_sig_encoding)

In [0]:
df['mode'] = df['mode'].map(mode_encoding)

In [0]:
# check worked out
df.dtypes

In [0]:
cols = list(df.columns)
cols = [cols[0]] + cols[0:]
df = df[cols]

In [0]:
df.head(1)

In [0]:
df['genre'].value_counts()

# MODELING: Nearest Neighbors
resources: https://scikit-learn.org/stable/modules/neighbors.html

In [0]:
neigh = NearestNeighbors()

In [0]:
# to remove the transformed columns from model 
# remove = ['key', 'mode','time_signature']
# features = [i for i in list(df.columns[4:]) if i not in remove]

features = list(df.columns[4:])

# target = 'track_id'

In [0]:
X = df[features]
# y = df[target]

X.shape # y.shape

In [0]:
neigh.fit(X) # NN doesn't need to fit Y

# Export Model with Joblib

In [0]:
filename = 'NearestNeighbor2.sav'

In [0]:
joblib.dump(neigh, filename)

In [0]:
# one hot encoding, rolled back
# def encode_and_bind(original_dataframe, feature_to_encode):
#     dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
#     res = pd.concat([original_dataframe, dummies], axis=1)
#     return(res)

# df = encode_and_bind(df, 'key')
# df = encode_and_bind(df, 'mode')
# df = encode_and_bind(df, 'time_signature')